<a href="https://colab.research.google.com/github/mhdSharuk/Data-Science-Bowl-2K19/blob/master/DSB_2019.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd
pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 28445)
import numpy as np
import os
import gc
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook
import json

In [0]:
specs = pd.read_csv('/content/drive/My Drive/specs.csv')
train_labels = pd.read_csv('./drive/My Drive/train_labels.csv.zip')
train = pd.read_csv('./drive/My Drive/train.csv.zip')
test = pd.read_csv('./drive/My Drive/test.csv.zip')
sub = pd.read_csv('./drive/My Drive/sample_submission.csv')

In [0]:
specs.head(6)

,event_id,info,args
0,2b9272f4,The end of system-initiated feedback (Correct)...,"[{""name"":""game_time"",""type"":""int"",""info"":""mill..."
1,df4fe8b6,The end of system-initiated feedback (Incorrec...,"[{""name"":""game_time"",""type"":""int"",""info"":""mill..."
2,3babcb9b,The end of system-initiated instruction event ...,"[{""name"":""game_time"",""type"":""int"",""info"":""mill..."
3,7f0836bf,The end of system-initiated instruction event ...,"[{""name"":""game_time"",""type"":""int"",""info"":""mill..."
4,ab3136ba,The end of system-initiated instruction event ...,"[{""name"":""game_time"",""type"":""int"",""info"":""mill..."
5,bbfe0445,The end of system-initiated instruction event ...,"[{""name"":""game_time"",""type"":""int"",""info"":""mill..."


In [21]:
train_labels.head(10)

,game_session,installation_id,title,num_correct,num_incorrect,accuracy,accuracy_group
0,6bdf9623adc94d89,0006a69f,Mushroom Sorter (Assessment),1,0,1.0,3
1,77b8ee947eb84b4e,0006a69f,Bird Measurer (Assessment),0,11,0.0,0
2,901acc108f55a5a1,0006a69f,Mushroom Sorter (Assessment),1,0,1.0,3
3,9501794defd84e4d,0006a69f,Mushroom Sorter (Assessment),1,1,0.5,2
4,a9ef3ecb3d1acc6a,0006a69f,Bird Measurer (Assessment),1,0,1.0,3
5,197a373a77101924,0006c192,Cauldron Filler (Assessment),1,0,1.0,3
6,957406a905d59afd,0006c192,Bird Measurer (Assessment),1,1,0.5,2
7,b2297d292892745a,0006c192,Mushroom Sorter (Assessment),0,4,0.0,0
8,ae691ec5ad5652cf,00129856,Bird Measurer (Assessment),1,0,1.0,3
9,7b536271e99518f0,001d0ed0,Bird Measurer (Assessment),0,5,0.0,0


In [68]:
train.groupby(['game_session','installation_id'])['title'].agg({'value_counts'}).rename(columns={'value_counts':'Total_no'}).head(50)

,,,Total_no
game_session,installation_id,title,
00000c4b5e2c400f,3ad8a814,Tree Top City - Level 1,1
000050630c4b081b,b57e4a95,Dino Drink,109
00005be8058d8e35,fb91172c,Balancing Act,1
0000b2837f00d9a4,100b0f77,Welcome to Lost Lagoon!,1
0000d11e30a5391c,cfabf670,Chow Time,130
0000d473b2f78883,856e7225,Sandcastle Builder (Activity),75
0000e00444c302d9,5dd7b740,Magma Peak - Level 1,1
00011d7f4b48ed4b,30fa1043,Crystal Caves - Level 3,1
00015b9a87c51eca,ab1cad03,Treasure Map,1


In [76]:
test.groupby(['game_session','installation_id'])['title'].agg({'value_counts'}).rename(columns={'value_counts':'Total_no'}).head(50)

,,,Total_no
game_session,installation_id,title,
000326e9d4516dd0,48166507,Fireworks (Activity),301
0003a9881ca100aa,13a0754c,Flower Waterer (Activity),96
00094643cae0f886,a6f65253,12 Monkeys,1
000ccfa39b7568d7,c42aeee6,Flower Waterer (Activity),31
001352e3c9cde9dd,7b728c89,Tree Top City - Level 2,1
001a139acd7fce92,285b65c8,Bird Measurer (Assessment),130
002566ee359d9118,87b899d9,Magma Peak - Level 1,1
00259719caa9a510,803be493,Egg Dropper (Activity),30
0026ec65d1714552,4360a0a2,Dino Dive,110


In [0]:
a = pd.DataFrame()

In [0]:
a = train[train['event_code']==4100]

In [96]:
a.head(100)

,event_id,game_session,timestamp,event_data,installation_id,event_count,event_code,game_time,title,type,world
2228,25fa8af4,901acc108f55a5a1,2019-08-06T05:22:32.357Z,"{""correct"":true,""stumps"":[1,2,4],""event_count""...",0006a69f,44,4100,31011,Mushroom Sorter (Assessment),Assessment,TREETOPCITY
2308,14de4c5d,80d34a30c2998653,2019-08-06T05:24:50.323Z,"{""distance"":10,""target_distances"":[5,6,7,8,9,1...",0006a69f,76,4100,114370,Air Show,Game,TREETOPCITY
2335,14de4c5d,80d34a30c2998653,2019-08-06T05:25:11.292Z,"{""distance"":9,""target_distances"":[5,6,7],""corr...",0006a69f,103,4100,135341,Air Show,Game,TREETOPCITY
2375,14de4c5d,80d34a30c2998653,2019-08-06T05:25:37.207Z,"{""distance"":3,""target_distances"":[5,6,7],""corr...",0006a69f,143,4100,161258,Air Show,Game,TREETOPCITY
2409,14de4c5d,80d34a30c2998653,2019-08-06T05:26:01.055Z,"{""distance"":8,""target_distances"":[5,6,7],""corr...",0006a69f,177,4100,185103,Air Show,Game,TREETOPCITY
2797,25fa8af4,6bdf9623adc94d89,2019-08-06T05:38:08.036Z,"{""correct"":true,""stumps"":[1,2,4],""event_count""...",0006a69f,30,4100,18026,Mushroom Sorter (Assessment),Assessment,TREETOPCITY
3725,25fa8af4,9501794defd84e4d,2019-08-06T20:35:12.290Z,"{""correct"":false,""stumps"":[3,2,5],""event_count...",0006a69f,29,4100,18484,Mushroom Sorter (Assessment),Assessment,TREETOPCITY
3734,25fa8af4,9501794defd84e4d,2019-08-06T20:35:16.846Z,"{""correct"":true,""stumps"":[2,3,5],""event_count""...",0006a69f,38,4100,23043,Mushroom Sorter (Assessment),Assessment,TREETOPCITY
3773,14de4c5d,8043825259dc7ddd,2019-08-06T20:36:23.582Z,"{""distance"":10,""target_distances"":[6,7,8,9,10]...",0006a69f,35,4100,46494,Air Show,Game,TREETOPCITY
3809,14de4c5d,8043825259dc7ddd,2019-08-06T20:36:54.289Z,"{""distance"":9,""target_distances"":[4,5,6,7,8,9,...",0006a69f,71,4100,77194,Air Show,Game,TREETOPCITY


In [0]:
train.head(6)

,event_id,game_session,timestamp,event_data,installation_id,event_count,event_code,game_time,title,type,world
0,27253bdc,45bb1e1b6b50c07b,2019-09-06T17:53:46.937Z,"{""event_code"": 2000, ""event_count"": 1}",0001e90f,1,2000,0,Welcome to Lost Lagoon!,Clip,NONE
1,27253bdc,17eeb7f223665f53,2019-09-06T17:54:17.519Z,"{""event_code"": 2000, ""event_count"": 1}",0001e90f,1,2000,0,Magma Peak - Level 1,Clip,MAGMAPEAK
2,77261ab5,0848ef14a8dc6892,2019-09-06T17:54:56.302Z,"{""version"":""1.0"",""event_count"":1,""game_time"":0...",0001e90f,1,2000,0,Sandcastle Builder (Activity),Activity,MAGMAPEAK
3,b2dba42b,0848ef14a8dc6892,2019-09-06T17:54:56.387Z,"{""description"":""Let's build a sandcastle! Firs...",0001e90f,2,3010,53,Sandcastle Builder (Activity),Activity,MAGMAPEAK
4,1bb5fbdb,0848ef14a8dc6892,2019-09-06T17:55:03.253Z,"{""description"":""Let's build a sandcastle! Firs...",0001e90f,3,3110,6972,Sandcastle Builder (Activity),Activity,MAGMAPEAK
5,1325467d,0848ef14a8dc6892,2019-09-06T17:55:06.279Z,"{""coordinates"":{""x"":583,""y"":605,""stage_width"":...",0001e90f,4,4070,9991,Sandcastle Builder (Activity),Activity,MAGMAPEAK


In [0]:
test.head(6)

,event_id,game_session,timestamp,event_data,installation_id,event_count,event_code,game_time,title,type,world
0,27253bdc,0ea9ecc81a565215,2019-09-10T16:50:24.910Z,"{""event_code"": 2000, ""event_count"": 1}",00abaee7,1,2000,0,Welcome to Lost Lagoon!,Clip,NONE
1,27253bdc,c1ea43d8b8261d27,2019-09-10T16:50:55.503Z,"{""event_code"": 2000, ""event_count"": 1}",00abaee7,1,2000,0,Magma Peak - Level 1,Clip,MAGMAPEAK
2,27253bdc,7ed86c6b72e725e2,2019-09-10T16:51:51.805Z,"{""event_code"": 2000, ""event_count"": 1}",00abaee7,1,2000,0,Magma Peak - Level 2,Clip,MAGMAPEAK
3,27253bdc,7e516ace50e7fe67,2019-09-10T16:53:12.825Z,"{""event_code"": 2000, ""event_count"": 1}",00abaee7,1,2000,0,Crystal Caves - Level 1,Clip,CRYSTALCAVES
4,7d093bf9,a022c3f60ba547e7,2019-09-10T16:54:12.115Z,"{""version"":""1.0"",""round"":0,""event_count"":1,""ga...",00abaee7,1,2000,0,Chow Time,Game,CRYSTALCAVES
5,f93fc684,a022c3f60ba547e7,2019-09-10T16:54:14.338Z,"{""coordinates"":{""x"":515,""y"":697,""stage_width"":...",00abaee7,2,4010,2232,Chow Time,Game,CRYSTALCAVES


In [61]:
sub.head(8)

,installation_id,accuracy_group
0,00abaee7,3
1,01242218,3
2,017c5718,3
3,01a44906,3
4,01bc6cb6,3
5,02256298,3
6,0267757a,3
7,027e7ce5,3
